In [30]:
import pandas as pd
import numpy as np
import requests
import pickle

In [2]:
yields_df = pd.read_csv('yields.csv', index_col=0)
yields_df = yields_df.iloc[1:] # drop first row which is just "YLD_YTM_MID"
yields_df.columns = [x.split()[0] for x in yields_df.columns] # remove ' GOVT'
yields_df.index = pd.to_datetime(yields_df.index)
yields_df

,912810ES3,912810ET1,912810EV6,912810EW4,912810EX2,912810EY0,912810EZ7,912810FA1,912810FB9,912810FE3,...,912803GF4,912803GJ6,912803GL1,912803GN7,912803GQ0,912803GT4,912803GV9,912803GX5,912803GZ0,912803HC0
2017-01-02,2.26,2.286,2.337,2.41,2.428,2.442,2.452,2.49,2.515,2.561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,2.264,2.291,2.335,2.408,2.429,2.442,2.451,2.491,2.513,2.558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,2.258,2.284,2.327,2.4,2.419,2.433,2.446,2.484,2.506,2.548,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-05,2.159,2.188,2.231,2.305,2.325,2.34,2.35,2.388,2.409,2.451,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-06,2.235,2.262,2.306,2.378,2.399,2.412,2.424,2.46,2.48,2.521,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-08,4.639,4.735,4.631,4.547,4.405,4.011,4.277,4.323,4.306,4.187,...,4.6,4.59,4.607,4.583,4.612,4.616,4.609,4.597,4.604,4.59
2024-07-09,4.679,4.725,4.644,4.555,4.402,4.065,4.271,4.328,4.308,4.199,...,4.623,4.61,4.625,4.602,4.63,4.633,4.622,4.612,4.62,4.606
2024-07-10,4.651,4.739,4.642,4.547,4.409,4.011,4.264,4.321,4.306,4.188,...,4.61,4.598,4.611,4.59,4.617,4.621,4.61,4.598,4.607,4.594
2024-07-11,4.572,4.663,4.535,4.431,4.295,3.911,4.16,4.213,4.2,4.086,...,4.543,4.534,4.547,4.527,4.555,4.56,4.549,4.537,4.55,4.538


In [26]:
def get_issue_date(cusip):
    print(cusip)
    url = 'https://www.treasurydirect.gov/TA_WS/securities/search?cusip={}&format=json'.format(cusip)
    r = requests.get(url)
    try:
        d = r.json()
        d = d[0] # first element is the only relevant dictionary
        return pd.to_datetime(d['issueDate'])
    except:
        # No data - Bad Issue Date
        print('Bad CUSIP')
        return None


In [25]:
get_issue_date('912803BD4')

912803BD4


IndexError: list index out of range

In [37]:
for date, yields in yields_df.iterrows():
    par_curve = {}
    for cusip in yields.index:
        info = security_search(cusip)
        tenor = date - pd.to_datetime(info['issueDate'])
        tenor = tenor.days / 365 # calculates the tenor, although in practice should use modified duration
        
        # creates a dictionary of the yield at a particular tenor
        par_curve[tenor] = yields[cusip]
        
    break

Bad CUSIP


TypeError: 'NoneType' object is not subscriptable

In [27]:
cusip_date_dict = {cusip: get_issue_date(cusip) for cusip in yields_df.columns}

912810ES3
912810ET1
912810EV6
912810EW4
912810EX2
912810EY0
912810EZ7
912810FA1
912810FB9
912810FE3
912810FF0
912810FG8
912810FJ2
912810FM5
912810FP8
912810FT0
912810PT9
912810PU6
912810PW2
912810PX0
912810QA9
912810QB7
912810QC5
912810QD3
912810QE1
912810QH4
912810QK7
912810QL5
912810QN1
912810QQ4
912810QS0
912810QT8
912810QU5
912810QW1
912810QX9
912810QY7
912810QZ4
912810RB6
912810RC4
912810RD2
912810RE0
912810RG5
912810RH3
912810RJ9
912810RK6
912810RM2
912810RN0
912810RP5
912810RQ3
912810RS9
912810RT7
912810RU4
912810RV2
912810RX8
912810RY6
912810RZ3
912810SA7
912810SC3
912810SD1
912810SE9
912810SF6
912810SH2
912810SJ8
912810SK5
912810SL3
912810SN9
912810SP4
912810SS8
912810SU3
912810SX7
912810SZ2
912810TB4
912810TD0
912810TG3
912810TJ7
912810TL2
912810TN8
912810TR9
912810TT5
912810TV0
912810TX6
912810UA4
912810SR0
912810SQ2
912810ST6
912810SW9
912810SY5
912810TA6
912810TC2
912810TF5
912810TH1
912810TK4
912810TM0
912810TQ1
912810TS7
912810TU2
912810TW8
912810TZ1
912810UB2
912828D56


In [31]:
with open('cusip_issue_date.pickle', 'wb') as handle:
    pickle.dump(cusip_date_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [39]:
cusip_date_dict_valid = {k:v for k, v in cusip_date_dict.items() if v}
yields_df_valid = yields_df[cusip_date_dict_valid.keys()]
yields_df_valid.head()

,912810ES3,912810ET1,912810EV6,912810EW4,912810EX2,912810EY0,912810EZ7,912810FA1,912810FB9,912810FE3,...,91282CJB8,91282CJE2,91282CJL6,91282CJS1,91282CJV4,91282CKB6,91282CKH3,91282CKK6,91282CKS9,91282CKY6
2017-01-02,2.26,2.286,2.337,2.41,2.428,2.442,2.452,2.49,2.515,2.561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,2.264,2.291,2.335,2.408,2.429,2.442,2.451,2.491,2.513,2.558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,2.258,2.284,2.327,2.4,2.419,2.433,2.446,2.484,2.506,2.548,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-05,2.159,2.188,2.231,2.305,2.325,2.34,2.35,2.388,2.409,2.451,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-06,2.235,2.262,2.306,2.378,2.399,2.412,2.424,2.46,2.48,2.521,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
pd.Series(cusip_date_dict_valid)

AttributeError: 'Series' object has no attribute 'reshape'